Программа повышения квалификации (научно-педагогических) работников НИУ ВШЭ

# Python для исследователей

*Татьяна Рогович, НИУ ВШЭ*

## Bag of Words (мешок слов), лемматизация и классификация текстов

Семинар

*Автор: Татьяна Рогович, НИУ ВШЭ*


In [1]:
import pandas as pd

### Что такое NLP?
Обработка естественного языка - (Natural Language Processing, NLP ), представляет собой набор методов для решения задач по обработке текстов. В этом уроке, мы попробуем загрузить и почистить текст новостей, а затем, использую простую модель признаков - мешка слов (Bag of Words), научимся довольно точно предсказывать фэйковая это новость или настоящая.

Но для начала посмотрим как работает bag of words на игрушечном примере.

### Создаем признаки (features) из Bag of Words

Как превратить текст в некоторое численное представление для машинного обучения? Один из таких подходов называется мешком слов (Bag of Words). Модель Bag of Words изучает все слова, которые есть в нашем корпусе, а затем моделирует каждый документ, подсчитывая количество раз, когда появляется каждое слово. Например, рассмотрим следующие два предложения:

Предложение 1: "The cat sat on the hat"

Предложение 2: "The dog ate the cat and the hat"

Для этих двух сообщений,получим следующий словарь:

{ the, cat, sat, on, hat, dog, ate, and }

Чтобы получить наш мешок слов, мы подсчитаем сколько раз каждое слово появилось в предложении. В предложение 1, "the" появилось два раза, а "cat", "sat", "on", и "hat" по одному разу каждый. Поэтому вектор признаков для этого предложения выглядит так:

{ the, cat, sat, on, hat, dog, ate, and }

{ 2, 1, 1, 1, 1, 0, 0, 0 }

Аналогично, признаки предложения 2 будут такими: 

{ 3, 1, 0, 0, 1, 1, 1, 1}

## Как перевести текст в признаки?

Если наша задача типовая, то скорее всего в sklearn уже есть библиотека, с помощью которой можно ее решить. Мы будем пользоваться классом CountVectorizer(), который при обучении создает признаки из всех слов обучающей выборки, при трансформации - подсчитывает встречание этих слов в тренировочной и тестовой выборке. А потом получившиеся наборы данных мы уже передаем алгоритмам для предсказания.

In [2]:
# создадим тренировочный список текстов
simple_train = ['call you tonight', 'Call me a cab', 'call 222-22-22', 'please call me.. please']

In [3]:
# Импортируем и иниициализируем класс CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [4]:
# обучаемся на данных
vect.fit(simple_train)

CountVectorizer()

In [5]:
# Смотрим сгенерированные признаки
vect.get_feature_names()


['22', '222', 'cab', 'call', 'me', 'please', 'tonight', 'you']

Обратите внимание, что в наш векторайзер уже зашита базовая обработка текста. Он приводит текст к нижнему регистру, и с помощью регулярок забирает только слова, состоящие из цифр и букв (знаки препинания удаляются), удаляет стоп-слова.

Метод fit здесь разбивает текст на токены и сохраняет их в "модель". Метод transform мы используем, чтобы создать разреженную матрицу, в которой мы будем хранить информацию, сколько каждое слово встречалось в каждом "тексте".

In [6]:
# преобразовываем исходный текст в матрицу признаков
simple_train_matrix = vect.transform(simple_train)
# каждый ряд - одно наблюдение (наш документ), каждая колонка - один признак (слово). 
# На пересечении - количество слов в документе.
simple_train_matrix

<4x8 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [7]:
simple_train_matrix.toarray()

array([[0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 1, 1, 1, 0, 0, 0],
       [2, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 2, 0, 0]], dtype=int64)

*sparse matrix*

Разреженная матрица. Хранит только координаты ненулевых значений. Сильно экономит место. 

*dense matrix*

Плотная матрица. Хранит все данные.
Если у вас матрица 1000 на 1000 из нулей - она все это хранит (1 мб ничего).

In [8]:
pd.DataFrame(simple_train_matrix.toarray(), columns=vect.get_feature_names(), index=simple_train)

22  222  cab  call  me  please  tonight  you
call you tonight          0    0    0     1   0       0        1    1
Call me a cab             0    0    1     1   1       0        0    0
call 222-22-22            2    1    0     1   0       0        0    0
please call me.. please   0    0    0     1   1       2        0    0

In [9]:
print('sparse matrix')
print(simple_train_matrix)

print('dense matrix')
print(simple_train_matrix.toarray())

sparse matrix
  (0, 3)	1
  (0, 6)	1
  (0, 7)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (2, 0)	2
  (2, 1)	1
  (2, 3)	1
  (3, 3)	1
  (3, 4)	1
  (3, 5)	2
dense matrix
[[0 0 0 1 0 0 1 1]
 [0 0 1 1 1 0 0 0]
 [2 1 0 1 0 0 0 0]
 [0 0 0 1 1 2 0 0]]


In [10]:
# создадим текст для теста и преобразуем обученным CountVectorizer
simple_test = ['Please don\'t call me, I will be busy']
simple_test_matrix = vect.transform(simple_test)
simple_test_matrix.toarray()

array([[0, 0, 0, 1, 1, 1, 0, 0]], dtype=int64)

In [11]:
# куда делся don't?
pd.DataFrame(simple_test_matrix.toarray(), columns=vect.get_feature_names(), index=simple_test)

22  222  cab  call  me  please  tonight  \
Please don't call me, I will be busy   0    0    0     1   1       1        0   

                                      you  
Please don't call me, I will be busy    0

Transform для тестовой выборки будет игнорировать токены, которые он не видел раньше (поэтому важен большой размер обучающей выборки и ее репрезентативность корпуса текстов).

### Стемминг (Stemming) и Лемматизация (Lemmatization)

Процесс стемминга заключается в преобразовании различных вариантов одного и того же слова в одну ядерную форму. Например у нас есть слова "running", "runs" и "run", которые семантически говорят об одном и том же, поэтому мы можем вместо них использовать одно слово "run". Стоит отметить, что в этом случае мы можем потерять некоторые грамматические признаки, например форму времени.

Воспользуемся алгоритмом Портера из модуля NLTK. Посмотрим на работу алгоритма на примере с вариантами слов run

In [17]:
!pip install nltk

You should consider upgrading via the 'c:\users\rogov\anaconda3\python.exe -m pip install --upgrade pip' command.


In [12]:
import nltk

In [13]:
stemmer = nltk.stem.PorterStemmer()

print("The stemmed form of running is: {}".format(stemmer.stem("running")))
print("The stemmed form of runs is: {}".format(stemmer.stem("runs")))
print("The stemmed form of run is: {}".format(stemmer.stem("run")))

The stemmed form of running is: run
The stemmed form of runs is: run
The stemmed form of run is: run


С русским языком не работает, к сожалению. Но сам Портер адаптировал алгоритм позже для ряда языков (смотри ниже Snowball stemming).

In [14]:
print("The stemmed form of бегать is: {}".format(stemmer.stem("бегать")))

The stemmed form of бегать is: бегать


Мы успешно обрезали наши слова до базовой формы. Алгоритм не использует баз основ слов, а лишь, применяя последовательно ряд правил, отсекает окончания и суффиксы, основываясь на особенностях языка, в связи с чем работает быстро, но не всегда безошибочно.

Однако у стемминга есть тенденция к "грубому" обрезанию концов слов. Например, для leaves мы получим следующее:

In [15]:
print("The stemmed form of leaves is: {}".format(stemmer.stem("leaves")))

The stemmed form of leaves is: leav


С одной стороны это нормально, но базовой формой слова будет leaf. Поэтому на помощь нам приходит лемматизация.

Лемматизация пытается достигнуть того же эффекта, но в отличие от стеммера, лемматизация использует реальный словарь слов и поэтому не будет обрубать окончания слов, а будет возвращать лемму.

Снова воспользуемся модулем NLTK и проверим, что он нам выдаст на leaves:

In [16]:
# Импортируем
from nltk.stem import WordNetLemmatizer
# Инициализируем 
lemm = WordNetLemmatizer()

print("The lemmatized form of leaves is: {}".format(lemm.lemmatize("leaves")))

The lemmatized form of leaves is: leaf


Получили верный ответ. Лемматизация обеспечивает нам более тонкую настройку по сравнению со стеммингом. Но чтобы интегрировать лемматизацию в count vectorizer, очистку текста придется делать сначала вручную (см. ниже).

## Работа с русскоязычными текстами

С поддержкой русского языка у nltk не супер, к сожалению. Можно воспользоваться алгоритмом для стемминга, который является адаптацией стеммера Портера (но с русским языком ему сильно тяжелее, чем с английским).

In [17]:
from nltk.stem.snowball import RussianStemmer

rus_stemmer = RussianStemmer()

print(rus_stemmer.stem('бежать'))
print(rus_stemmer.stem('бегущий'))
print(rus_stemmer.stem('бегающий'))

бежа
бегущ
бега


Если библиотека pymorphy, которая делает лемматизацию, основываясь на словаре проекта OpenCorpora. https://pymorphy2.readthedocs.io/

Далее объяснения и примеры из документации.

В pymorphy2 для морфологического анализа слов (русских) есть класс MorphAnalyzer.
Экземпляры класса MorphAnalyzer занимают порядка 10-15Мб оперативной памяти (т.к. загружают в память словари, данные для предсказателя и т.д.); старайтесь ораганизовать свой код так, чтоб создавать экземпляр MorphAnalyzer заранее и работать с этим единственным экземпляром в дальнейшем.

Метод MorphAnalyzer.parse() принимает слово (обязательно в нижнем регистре) и возвращает все возможные разборы слова:

In [18]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
morph.parse('стали')

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.984662, methods_stack=((<DictionaryAnalyzer>, 'стали', 904, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 9),))]

Обратите внимание, что в этом примере слово “стали” может быть разобрано и как глагол (“они стали лучше справляться”), и как существительное (“кислородно-конверторный способ получения стали”). На основе одной лишь информации о том, как слово пишется, понять, какой разбор правильный, нельзя, поэтому анализатор может возвращать несколько вариантов разбора.

**Выбор правильного разбора**  
pymorphy2 возвращает все допустимые варианты разбора, но на практике обычно нужен только один вариант, правильный.

У каждого разбора есть параметр score. Score - это оценка P(tag|word), оценка вероятности того, что данный разбор правильный.

pymorphy2 умеет разбирать не только словарные слова; для несловарных слов автоматически задействуется предсказатель. Например, попробуем разобрать слово “бутявковедами” - pymorphy2 поймет, что это форма творительного падежа множественного числа существительного “бутявковед”, и что “бутявковед” - одушевленный и мужского рода:

In [27]:
morph.parse('бутявковедами')

[Parse(word='бутявковедами', tag=OpencorporaTag('NOUN,anim,masc plur,ablt'), normal_form='бутявковед', score=1.0, methods_stack=((<FakeDictionary>, 'бутявковедами', 52, 10), (<KnownSuffixAnalyzer>, 'едами')))]

У каждого разбора есть нормальная форма, которую можно получить, обратившись к атрибуту normal_form.

In [19]:
p = morph.parse('стали')[0]
p.normal_form

'стать'

pymorphy2 умеет склонять (ставить в какую-то другую форму) слова. Чтобы просклонять слово, его нужно сначала разобрать - понять, в какой форме оно стоит в настоящий момент:

In [22]:
butyavka = morph.parse('бутявки')[0]
butyavka.normal_form

'бутявка'

In [24]:
butyavka.inflect({'gent'})[0] # родительный падеж

'бутявки'

С помощью атрибута lexeme можно получить лексему слова:



In [31]:
butyavka.lexeme

[Parse(word='бутявка', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='бутявка', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'явка', 8, 0), (<UnknownPrefixAnalyzer>, 'бут'))),
 Parse(word='бутявки', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='бутявка', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'явки', 8, 1), (<UnknownPrefixAnalyzer>, 'бут'))),
 Parse(word='бутявке', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='бутявка', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'явке', 8, 2), (<UnknownPrefixAnalyzer>, 'бут'))),
 Parse(word='бутявку', tag=OpencorporaTag('NOUN,inan,femn sing,accs'), normal_form='бутявка', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'явку', 8, 3), (<UnknownPrefixAnalyzer>, 'бут'))),
 Parse(word='бутявкой', tag=OpencorporaTag('NOUN,inan,femn sing,ablt'), normal_form='бутявка', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'явкой', 8, 4), (<UnknownPrefixAnalyzer>, 'бут'))),
 Parse(word='бутявкою', tag=

Теги и граммемы в pymorphy2 записываются латиницей (например, NOUN). Но часто удобнее использовать кириллические названия граммем (например, СУЩ вместо NOUN). Чтобы получить тег в виде строки, записанной кириллицей, используйте свойство OpencorporaTag.cyr_repr:

In [25]:
print(p)
print(p.tag)
print(p.tag.cyr_repr)


Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.984662, methods_stack=((<DictionaryAnalyzer>, 'стали', 904, 4),))
VERB,perf,intr plur,past,indc
ГЛ,сов,неперех мн,прош,изъяв


Давайте попробуем преобразовать предложение в список нормализованных слов.

In [26]:
test = 'Лемматизация – это процесс преобразования слова в его базовую форму. Разница между стемминг (stemming) и лемматизацией заключается в том, что лемматизация учитывает контекст и преобразует слово в его значимую базовую форму, тогда как стемминг просто удаляет последние несколько символов, что часто приводит к неверному значению и орфографическим ошибкам.'

In [27]:
import re
test_words = [word.lower() for word in re.findall(r'[а-яА-Я]+', test)]

In [28]:
test_morph = [morph.parse(word)[0].normal_form for word in test_words]

In [32]:
for word, form in zip(test_words, test_morph):
    if word != form:
        print(word, '->', form)
    else:
        print(word)

лемматизация
это
процесс
преобразования -> преобразование
слова -> слово
в
его -> он
базовую -> базовый
форму -> форма
разница
между
стемминг
и
лемматизацией -> лемматизация
заключается -> заключаться
в
том -> тот
что
лемматизация
учитывает -> учитывать
контекст
и
преобразует -> преобразовать
слово
в
его -> он
значимую -> значимый
базовую -> базовый
форму -> форма
тогда
как
стемминг
просто
удаляет -> удалять
последние -> последний
несколько
символов -> символ
что
часто
приводит -> приводить
к
неверному -> неверный
значению -> значение
и
орфографическим -> орфографический
ошибкам -> ошибка


## Стоп-слова

In [33]:
import nltk
nltk.download("stopwords")  
# Скачиваем нужные нам наборы. Необязательно скачивать все. Нам пока что нужен только
# corpora stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rogov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Теперь мы можем использовать nltk для получения списка стоп-слов:

In [34]:
from nltk.corpus import stopwords # Импортируем список стол-слов
print(stopwords.words("english")) # Стоп-слова для английского языка

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [35]:
print(stopwords.words("russian")) # Стоп-слова для русского языка

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

# Are those news fake or real?
У нас есть датасет состоящий из заголовка новости, текста новости и лейбла, который показывает фейковая это новость или реальная.

Нашей задачей будет натренировать модель, чтобы она могла определить фейковая новость или реальная.

### Загружаем данные

In [36]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/rogovich/Data/master/data/fake_or_real_news.csv", header=0)

In [37]:
data.head()

Unnamed: 0                                              title  \
0        8476                       You Can Smell Hillary’s Fear   
1       10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2        3608        Kerry to go to Paris in gesture of sympathy   
3       10142  Bernie supporters on Twitter erupt in anger ag...   
4         875   The Battle of New York: Why This Primary Matters   

                                                text label  
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
2  U.S. Secretary of State John F. Kerry said Mon...  REAL  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE  
4  It's primary day in New York and front-runners...  REAL

In [38]:
del data['Unnamed: 0']

In [39]:
data.columns

Index(['title', 'text', 'label'], dtype='object')

Три основных столбца называются "title", "text", и "label".

Разделим наши данные на обучающую и тестовую выборки, чтобы потом можно было проверить насколько качественно работает наш алгоритм. За y обозначим наш лейбл. За X саму колонку с текстом новостей.

In [40]:
y = data['label']
X = data[['text']] # двойные скобки, чтобы она осталась датафреймом

Для разделения данных на тестовую и обучающую выборки, воспользуемся функцией train_test_split из модуля sklearn. Как видно из названия она делит выборку на части и размещает данные по выбранным массивам. Test_size указывает на то, какую часть выборки нужно отложить для теста.

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33,random_state=53)

Теперь, когда мы разбили данные и получили обучающую выборку, посмотрим как выглядит текст новости. Выведем любую новость

In [42]:
X_train.shape

(4244, 1)

In [43]:
X_test.shape

(2091, 1)

In [44]:
print(X_train.iloc[1,0])

Report Copyright Violation Do you think there will be as many doom sayers if trump should get in office ? I notice here at GLP the amount of doom sayers seems to go down when a republican is in office (Bush). But when the left get in office the doomsaying increases. Now i am sure the effect is opposite. If trump gets in office i am sure the doomsaying will increase on the left side of the political spectrum. Page 1


Иногда внутри текста мы можем иногда увидить HTML тэги (например "br"), аббревиатуры, пунктуацию, которые являются распространенными проблемы при обработке текста из Интернета. Но вроде в наших данных этого нет.

# Продолжим с новостями

Подробнее документацию CountVectorizer можно почитать тут:
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [45]:
# Инициализируем объект "CountVectorizer", метод для работы с bag of words
# из scikit-learn
vectorizer = CountVectorizer(analyzer = "word",   \
                             stop_words = 'english',   \
                             max_features = 1000) 
                                # max_features = ограничиваем максимальное
                                # количество слов-признаков для ускорения
                                # работы алгоритма. Если не ограничивать -
                                # количество признаков будет равно всем
                                # уникальным словам в нашем корпусе выборки

# fit_transform() делает две вещи: Сначала он фитит модель
# и изучает словарь; Потом трансформирует нашу обучающую выборку
# в вектор признаков В fit_transform мы передаем список слов
train_data_features = vectorizer.fit_transform(X_train['text'])

# Используем удобные массивы из Numpy
train_data_features = train_data_features.toarray()

Посмотрим как выглядят наши обучающие данные:

In [46]:
print(train_data_features.shape)

(4244, 1000)


In [47]:
train_data_features[:10]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 5, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

В ней 4244 строк и 1000 признаков (по одному на каждое слово в словаре).

Теперь наша модель Bag of Words натренирована, посмотрим на получившийся словарь:

In [48]:
vocab = vectorizer.get_feature_names()
print(vocab)

['000', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18', '20', '2008', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '22', '24', '25', '26', '27', '28', '30', '40', '50', '60', 'ability', 'able', 'abortion', 'access', 'according', 'account', 'accused', 'act', 'action', 'actions', 'activists', 'actually', 'added', 'adding', 'address', 'administration', 'african', 'age', 'agency', 'agenda', 'agents', 'ago', 'agree', 'agreement', 'ahead', 'air', 'al', 'allies', 'allow', 'allowed', 'america', 'american', 'americans', 'announced', 'answer', 'anti', 'appear', 'appeared', 'appears', 'approach', 'april', 'arabia', 'area', 'areas', 'aren', 'armed', 'army', 'article', 'ask', 'asked', 'assad', 'associated', 'attack', 'attacks', 'attempt', 'attention', 'attorney', 'author', 'authorities', 'authority', 'available', 'average', 'away', 'backed', 'bad', 'ballot', 'bank', 'banks', 'barack', 'base', 'based', 'battle', 'began', 'begin', 'beginning', 'believe', 'believes', 'benefits'

Дополнительно, можно вывести сколько раз слово встречается в словаре:

In [25]:
import numpy as np

# Подсчитаем количество слов в наших данных
dist = np.sum(train_data_features, axis=0)

# Выведем для каждого слова его количество
for tag, count in zip(vocab, dist):
    print(count, tag)

1694 000
1421 10
548 100
1006 11
570 12
357 13
380 14
762 15
404 16
363 17
394 18
779 20
628 2008
355 2010
428 2011
907 2012
539 2013
743 2014
862 2015
2681 2016
344 22
342 24
557 25
389 26
384 27
394 28
708 30
441 40
564 50
330 60
424 ability
859 able
398 abortion
598 access
2089 according
413 account
345 accused
1114 act
826 action
529 actions
331 activists
1161 actually
854 added
334 adding
566 address
1504 administration
497 african
394 age
609 agency
376 agenda
392 agents
1055 ago
365 agree
582 agreement
485 ahead
777 air
957 al
461 allies
497 allow
413 allowed
2549 america
3664 american
2170 americans
652 announced
501 answer
1079 anti
384 appear
421 appeared
436 appears
395 approach
374 april
373 arabia
586 area
408 areas
377 aren
375 armed
511 army
747 article
471 ask
1197 asked
392 assad
461 associated
1257 attack
1128 attacks
358 attempt
516 attention
471 attorney
453 author
495 authorities
360 authority
364 available
418 average
1147 away
369 backed
711 bad
360 ballot
503 ba

1472 today
2673 told
1016 took
437 total
394 town
792 trade
543 tried
436 troops
813 true
14641 trump
380 trust
604 truth
605 try
973 trying
1226 tuesday
349 turkey
671 turn
571 turned
430 tv
767 twitter
374 ukraine
592 understand
560 union
2534 united
1026 university
402 usa
1683 use
1450 used
783 using
333 various
1639 ve
372 veterans
578 vice
731 victory
990 video
575 view
439 views
723 violence
360 virginia
2392 vote
421 voted
460 voter
2526 voters
795 votes
1056 voting
1001 wall
2076 want
573 wanted
683 wants
2822 war
370 wars
1978 washington
553 wasn
557 watch
718 water
2923 way
518 ways
648 weapons
331 website
968 wednesday
1853 week
697 weeks
781 went
682 west
632 western
3089 white
462 wife
587 wikileaks
364 willing
1444 win
371 wing
524 winning
342 wins
740 woman
1652 women
1381 won
427 word
691 words
1803 work
516 worked
577 workers
1123 working
3328 world
414 worse
358 worth
402 wouldn
561 wrong
897 wrote
3384 year
3452 years
512 yes
1506 york
743 young


### Попробуем построить логистическую модель
В данный момент у нас есть численные значения признаков из обучающей выборки из Bag of Words и оригинальные лейблы новости, поэтому займемся обучением с учителем. Мы будем использовать классификатор логистическую регрессию. Этот алгоритм включен в scikit-learn. Я не буду вдавать в подробности работы алгоритма, здесь мне важно показать, что такая структура данных

In [49]:
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression(max_iter = 1000)

# Учим алгоритм предсказывает класс новости - fake или real
reg.fit(train_data_features, y_train)

LogisticRegression(max_iter=1000)

### Делаем предсказание
Осталось только запустить подготовленный алгоритм на нашей тестовой выборке и проверить результаты предсказания.

Стоит отметить, что для тестовой выборки мы уже используем метод "transform", а не "fit_transform" в Bag of Words.

Но для начала преобразуем нашу тестовую выборку в bag of words.

In [50]:
# очищаем тестовую подвыбоку

# Конвертируем получившиеся bag of words в массив Numpy
test_data_features = vectorizer.transform(X_test['text'])
test_data_features = test_data_features.toarray()

# Делаем предсказание логистической регрессией
result = reg.predict(test_data_features)

Теперь проверим насколько точны были предсказания. Accuracy_score - находит долю правильно предсказанных классов.

In [51]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, result)

0.866571018651363

86% - Отличный результат.

### TfidfVectorizer

В больших текстах, некоторые слова могут встречаться очень часто (например “the”, “a”, “is” в аглийском) при этом неся мало информации об истинном содержании текста. Поэтому мы можем добавить веса словами.

Tf-idf означает "frequency-inverse document frequency" (частотно-обратная частота документа) и представляет собой статистическую меру, используемую для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса. Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции. 

Большой вес в TF-IDF получат слова с высокой частотой в пределах конкретного документа и с низкой частотой употреблений в других документах.


{\displaystyle \mathrm {tf} (t,d)={\frac {n_{t}}{\sum _{k}n_{k}}}} ,


https://ru.wikipedia.org/wiki/TF-IDF



In [76]:
import math
example1 = 'в этом документе десять слов и одно из них заяц'.split()
example2 = 'а в этом такого слова нет'.split()
example3 = 'а здесь слова заяц снова есть есть заяц в этом документе'.split()

print('tf слова "заяц" в первом документе', example1.count('заяц')/len(example1))
print('tf слова "заяц" во втором документе', example2.count('заяц')/len(example2))
print('tf слова "заяц" во третьем документе', example3.count('заяц')/len(example3))

print('idf для слова заяц', math.log10(3/2))

print('tf*idf слова "заяц" в первом документе', example1.count('заяц')/len(example1) * math.log10(3/2))
print('tf*idf слова "заяц" во втором документе', example2.count('заяц')/len(example2) * math.log10(3/2))
print('tf*idf слова "заяц" во третьем документе', example3.count('заяц')/len(example3) * math.log10(3/2))

tf слова "заяц" в первом документе 0.1
tf слова "заяц" во втором документе 0.0
tf слова "заяц" во третьем документе 0.18181818181818182
idf для слова заяц 0.17609125905568124
tf*idf слова "заяц" в первом документе 0.017609125905568124
tf*idf слова "заяц" во втором документе 0.0
tf*idf слова "заяц" во третьем документе 0.032016592555578406


Используем Tfidvectorizer для конвертации данных в веса tfidf.

In [52]:
# Импортируем TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Инициализируем объект TfidfVectorizer: tfidf_vectorizer
# А также ограничили выдачу слов, которые мало встречаются в текстах, 
# использовав параметр max_df
tfidf_vectorizer = TfidfVectorizer(max_df = 0.7, min_df = 0.2) 
# Ограничиваем минимальный и максимальный порог встречаемости слова в текстах

Натренируем на тех же тестовых данных, что и CountVectorizer

In [53]:
# Тренируем классификатор
tfidf_train = tfidf_vectorizer.fit_transform(X_train['text'])
tfidf_test = tfidf_vectorizer.transform(X_test['text'])

Посмотрим что получили

In [54]:
# Выведем 10 первых признаков
print(tfidf_vectorizer.get_feature_names()[:5])

# Выведем первые 5 векторов обучающей выборки
print(tfidf_train.A[:5])


['10', '2016', 'about', 'according', 'after']
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.14373782 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.24683315 ... 0.         0.         0.        ]
 [0.         0.         0.06972137 ... 0.         0.16109609 0.        ]]


In [55]:
len(tfidf_vectorizer.get_feature_names()) # всего слов, удовлетворяющим нашим порогам было 212

212

Посмотрим, повлияет ли изменение метрики на качество алгоритма.

In [62]:
# Конвертируем получившиеся bag of words в массив Numpy

reg.fit(tfidf_train, y_train)
# Делаем предсказание логистической регрессией
result = reg.predict(tfidf_test)
accuracy_score(y_test, result)

0.8493543758967002

Немного ухудшилось. Тут на самом деле играет роль то, что с данными в таком виде лучше работают другие алгоритмы, но это уже за пределами нашего курса.

### Confusion Matrix

Для более наглядного примера, мы можем построить Confusion Matrix. Диагональные элементы матрицы указывают на количество верных попаданий для данного класса (Fake или Real в нашем случае). Остальные элементы в строке показывают сколько новостей ушли в другой класс.

In [64]:
from sklearn import metrics
metrics.confusion_matrix(y_test, result)

array([[866, 142],
       [173, 910]], dtype=int64)

142 фейковых новостей были некорректно отмечены как правдивые. И 173 реальных новостей были отмечены как фейковые.

Эта история важна, когда цена ошибки на одном из классов более высока. Например, наша алгоритм чаще называл фейковые новости реальными, чем наоборот. Если нам важно не назвать реальную новость фейковой, то мы ошиблись только 173 раз. А вот если наоборот, то цена ошибки выше.